In [35]:
from zotero_notion_sync import config
import requests
from typing import Optional
import json

In [34]:
# Zotero
zotero_headers = {"Zotero-API-Key": config.ZOTERO_API_KEY}
zotero_base_url = f"https://api.zotero.org/users/{config.ZOTERO_USER_ID}"
zotero_params = {"format": "json"}

# Notion
notion_headers = {
    "Authorization": f"Bearer {config.NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}
notion_db_id = config.NOTION_DATABASE_ID
notion_base_url = "https://api.notion.com/v1"
# Notion fields name
nt_title = "Title"
nt_collections = "Collections"
nt_authors = "Authors"
nt_source_url = "Source URL"
nt_tags = "Tags"
nt_item_type = "Item Type"
nt_publisher = "Publisher"
nt_extra = "Extra"
nt_doi = "DOI"
nt_abstract = "Abstract"
nt_status = "Status"
nt_category = "Category"
nt_date_accessed = "Date Accessed"
nt_publication_date = "Publication Date"
nt_modified_date = "Modified Date"

In [25]:
# Fetch Zotero references

zotero_items_url = zotero_base_url + "/items"

response = requests.get(
    zotero_items_url, headers=zotero_headers, params=zotero_params, timeout=30
)
response_data = response.json()

In [20]:
response.headers

# `rel="next"` exists in the headers, this means there are more than one response url

{'Date': 'Mon, 24 Feb 2025 10:50:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '12585', 'Connection': 'keep-alive', 'Server': 'Apache/2.4.62 ()', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Zotero-API-Version': '3', 'Zotero-Schema-Version': '28', 'Total-Results': '110', 'Link': '<https://api.zotero.org/users/15574376/items?start=25>; rel="next", <https://api.zotero.org/users/15574376/items?start=100>; rel="last", <https://www.zotero.org/users/15574376/items>; rel="alternate"', 'Last-Modified-Version': '528', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'}

In [26]:
next_url = zotero_items_url
zotero_references = []


while next_url:
    response = requests.get(
        next_url,
        headers=zotero_headers,
        params=zotero_params,
        timeout=30
    )
    
    response_data = response.json()
    zotero_references.extend(response_data)
    
    if "Link" in response.headers:
        links = response.headers["Link"].split(', ')
        next_url = None
    
        for link in links:
            if 'rel="next"' in link:
                next_url = link.split(';')[0].strip("<>")
                break
    
    else:
        next_url = None
    
    # print("Current fetched count: %s", len(zotero_references))
    
zotero_references

[{'key': 'LUHNJPE5',
  'version': 528,
  'library': {'type': 'user',
   'id': 15574376,
   'name': 'monireachtang',
   'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang',
     'type': 'text/html'}}},
  'links': {'self': {'href': 'https://api.zotero.org/users/15574376/items/LUHNJPE5',
    'type': 'application/json'},
   'alternate': {'href': 'https://www.zotero.org/monireachtang/items/LUHNJPE5',
    'type': 'text/html'}},
  'meta': {'creatorSummary': 'ASEAN', 'parsedDate': '2020', 'numChildren': 0},
  'data': {'key': 'LUHNJPE5',
   'version': 528,
   'itemType': 'webpage',
   'title': 'ASEAN Digital Masterplan 2025',
   'creators': [{'creatorType': 'author', 'name': 'ASEAN'}],
   'abstractNote': '',
   'websiteTitle': 'ASEAN Main Portal',
   'websiteType': '',
   'date': '2020',
   'shortTitle': '',
   'url': 'https://asean.org/book/asean-digital-masterplan-2025/',
   'accessDate': '2024-12-12T16:41:11Z',
   'language': 'en-US',
   'rights': '',
   'extra': '',
   't

In [30]:
# Fetch zotero collections

zotero_collections_url = zotero_base_url + "/collections"
zotero_collections = {}

def fetch_zotero_collections():
    base_url = zotero_collections_url
    
    response = requests.get(
        base_url,
        headers=zotero_headers,
        params=zotero_params,
        timeout=30,
    )
    
    response_data = response.json()
    
    # print(response_data)
    # # Output: [{'key': 'ZY4P7TXF', 'version': 281, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/ZY4P7TXF', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/ZY4P7TXF', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 2}, 'data': {'key': 'ZY4P7TXF', 'version': 281, 'name': 'Family', 'parentCollection': False, 'relations': {}}}, {'key': 'AZYQMZYU', 'version': 214, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/AZYQMZYU', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/AZYQMZYU', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 9}, 'data': {'key': 'AZYQMZYU', 'version': 214, 'name': 'Academic Writing', 'parentCollection': False, 'relations': {}}}, {'key': 'EPGK5DFF', 'version': 156, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/EPGK5DFF', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/EPGK5DFF', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 33}, 'data': {'key': 'EPGK5DFF', 'version': 156, 'name': 'Philosophy of Technology', 'parentCollection': False, 'relations': {}}}, {'key': 'VUS9AZXW', 'version': 56, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/VUS9AZXW', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/VUS9AZXW', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 10}, 'data': {'key': 'VUS9AZXW', 'version': 56, 'name': 'Public Sector Innovation', 'parentCollection': False, 'relations': {}}}, {'key': 'MXS9X79T', 'version': 54, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/MXS9X79T', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/MXS9X79T', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 18}, 'data': {'key': 'MXS9X79T', 'version': 54, 'name': 'Tech Transfer', 'parentCollection': False, 'relations': {}}}, {'key': 'X7AY75N7', 'version': 53, 'library': {'type': 'user', 'id': 15574376, 'name': 'monireachtang', 'links': {'alternate': {'href': 'https://www.zotero.org/monireachtang', 'type': 'text/html'}}}, 'links': {'self': {'href': 'https://api.zotero.org/users/15574376/collections/X7AY75N7', 'type': 'application/json'}, 'alternate': {'href': 'https://www.zotero.org/monireachtang/collections/X7AY75N7', 'type': 'text/html'}}, 'meta': {'numCollections': 0, 'numItems': 38}, 'data': {'key': 'X7AY75N7', 'version': 53, 'name': 'AI', 'parentCollection': False, 'relations': {}}}]
    
    if response.status_code == 200 and response_data:
        for collection in response_data:
            if(
                isinstance(collection, dict)
                and "key" in collection
                and "data" in collection
                and isinstance(collection["data"], dict)
                and "name" in collection["data"]
            ):
                zotero_collections[collection["key"]] = collection["data"]["name"]
    else:
        print("Some Errors Exist.")

fetch_zotero_collections()
zotero_collections

{'ZY4P7TXF': 'Family',
 'AZYQMZYU': 'Academic Writing',
 'EPGK5DFF': 'Philosophy of Technology',
 'VUS9AZXW': 'Public Sector Innovation',
 'MXS9X79T': 'Tech Transfer',
 'X7AY75N7': 'AI'}

In [36]:
# Add reference to notion

def find_matching_notion_page_id(title: str, zotero_collection_names: list) -> Optional[str]:
    
    notion_search_url: str = f"{notion_base_url}/databases/{notion_db_id}/query"
    
    # Construction filters for each zotero_collection_names
    collection_filters: list = []
    
    if zotero_collection_names:
        collection_filters = [
            {
                "property": nt_collections,
                "multi_select": {"contains": name}
            } for name in zotero_collection_names if name 
        ]
    # Build the payload to check for a title match and at least one collection match
    search_payload = {
        "filter": {
            "and": [
                {
                    "property": nt_title,
                    "title": {"equals": title}
                }
            ]
        }
    }

    # Add the collection_filters only if it is not empty, since the "or" expects a non-empty list of filters
    if collection_filters:
        search_payload["filter"]["and"].append({"or": collection_filters})
        
    try:
        response = requests.post(
            notion_search_url,
            headers=notion_headers,
            data=json.dumps(search_payload),
            timeout=30,
        )
        response_data = response.json()
    except ValueError:
        return None
    except Exception as e:
        raise
    
    if response.status_code == 200 and response_data:
        # Check if the result is a dictionary and has the required key
        if (
            isinstance(response_data, dict)
            and "results" in response_data
            and isinstance(response_data["results"], list)
        ):
            results = response_data["results"]
        
        if len(results) > 1:
            print(
                "Multiple entries found for title: '%s' and collections: '%s'. Returning the first match.",
                title, 
                zotero_collection_names
            )
            
        # If a result is found, return the page ID for updating the notion database
        if results:
            return results[0]["id"]
        
    return None

def format_authors(creators: list) -> Optional[str]:
    ...

def parse_date(date_str: str) -> Optional[str]:
    return None

def process_abstract(reference: dict) -> Optional[str]:
    pass

def format_zotero_collections(reference: dict, zotero_collections: dict) -> list:
    pass

# Functions for Notion Data Properties
def notion_title(reference: list) -> dict:
    return {
        "title": [
            {
                "text": {
                    "content": reference["data"]["title"]
                }
            }
        ]
    }

def notion_collections(zotero_collection_names: list) -> dict:
    return {
        "multi_select": [
            {
                "name": collection
            }
            for collection in zotero_collection_names if zotero_collection_names
        ]
    }

def notion_authors(authors: str) -> dict:
    ...

def notion_source_url(reference: dict) -> dict:
    ...

def notion_tags(reference: dict) -> dict:
    ...

def notion_item_type(reference: dict) -> dict:
    ...

def notion_publisher(reference: dict) -> dict:
    ...

def notion_extra(reference: dict) -> dict:
    ...

def notion_doi(reference: dict) -> dict:
    ...

def notion_abstract(abstract: str) -> dict:
    pass

def notion_status():
    pass

def notion_category():
    pass

def notion_access_date(access_date: str) -> dict:
    pass

def notion_publication_date(publication_date: str) -> dict:
    ...
    
def notion_modified_date(modified_date: str) -> dict:
    pass

def sync_to_notion(reference: dict, zotero_collections: dict) -> None:
    # Convert the authors to a comma-separated string required by Notion's multi_select field, Zotero gives a list of dictionaries instead (example: `'creators': [ { 'creatorType': 'author', 'firstName': 'Qiulin', 'lastName': 'Chen' }, { 'creatorType': 'author', 'firstName': 'Duo', 'lastName': 'Xu' }, { 'creatorType': 'author', 'firstName': 'Yi', 'lastName': 'Zhou' } ]`)
    authors: Optional[str] = format_authors()
    
    # Convert the dynamic zotero data to notion-compatible one ('YYYY-MM-DD' or 'YYYY-MM-DD HH:mm:ss'). Zotero's date can take different forms: `"%Y-%m-%dT%H:%M:%S%z",`, `"%Y-%m-%d"`, `"%Y/%m"`, or `"%Y"`
    access_date: Optional[str] = parse_date()
    publication_date: Optional[str] = parse_date()
    modified_date: Optional[str] = parse_date()
    
    # Process abstract (truncate to 2000 characters if too long)
    abstract: Optional[str] = process_abstract()
    
    # Convert the zotero_collections dict to a list of collection names
    zotero_collection_names: list = format_zotero_collections()
    
    # Prepare data for notion
    notion_data_parent: dict = {"database_id": notion_db_id}
    
    notion_data_properties_title: dict = notion_title(reference=reference)
    notion_data_properties_collections: dict = notion_collections(zotero_collection_names=zotero_collection_names)
    notion_data_properties_authors: dict = notion_authors(authors=authors)
    notion_data_properties_source_url: dict = notion_source_url(reference=reference)
    notion_data_properties_tags: dict = notion_tags(reference=reference)
    notion_data_properties_item_type: dict = notion_item_type(reference=reference)
    notion_data_properties_publisher: dict = notion_publisher(reference=reference)
    notion_data_properties_extra: dict = notion_extra(reference=reference)
    notion_data_properties_doi: dict = notion_doi(reference=reference)
    notion_data_properties_abstract: dict = notion_abstract(abstract=abstract)
    notion_data_properties_status: dict = notion_status()
    notion_data_properties_category: dict = notion_category()
    notion_data_properties_access_date: dict = notion_access_date(access_date=access_date)
    notion_data_properties_publication_date: dict = notion_publication_date(publication_date=publication_date)
    notion_data_properties_modified_date: dict = notion_modified_date(modified_date=modified_date)

    data_for_notion = {
        "parent": notion_data_parent,
        "properties": {
            nt_title: notion_data_properties_title,
            nt_collections: notion_data_properties_collections,
            nt_authors: notion_data_properties_authors,
            nt_source_url: notion_data_properties_source_url,
            nt_tags: notion_data_properties_tags,
            nt_item_type: notion_data_properties_item_type,
            nt_publisher: notion_data_properties_publisher,
            nt_extra: notion_data_properties_extra,
            nt_doi: notion_data_properties_doi,
            nt_abstract: notion_data_properties_abstract,
            nt_status: notion_data_properties_status,
            nt_category: notion_data_properties_category,
            nt_date_accessed: notion_data_properties_access_date,
            nt_publication_date: notion_data_properties_publication_date,
            nt_modified_date: notion_data_properties_modified_date,
        }
    }

# Main function

def update_reference_in_notion(notion_page_id: str, reference: dict, zotero_collection_names: list) -> None:
    pass

def add_reference_to_notion(reference: dict, zotero_collections: dict) -> None:
    
    # Check if the reference already exists in Notion by retrieving its ID (a.k.a., page ID)
    notion_page_id: Optional[str] = find_matching_notion_page_id()
    
    if notion_page_id:
        update_reference_in_notion(notion_page_id=notion_page_id, reference=reference, zotero_collection_names=zotero_collection_names)
        return


    
    
    
    
    
    

In [38]:
find_matching_notion_page_id(
    title="AI’s Effects on Economic Growth in Aging Society: Induced Innovation and Labor Supplemental Substitution",
    zotero_collection_names=['AI']
)